Author: [Mélanie Gornet](https://github.com/mgornet)

<div style="font-size:22pt; line-height:25pt; font-weight:bold; text-align:center;">Supervised Learning of Spiking Neural Networks (SNNs)</div><br>

This notebook was written as an evaluation project for the Algorithms in Machine Learning class at ISAE-SUPAERO.

This notebook is mainly adapted from a work by [Romain Zimmer](https://github.com/romainzimmer) and his team:
>[1] R. Zimmer, T. Pellegrini2 , S. F. Singh, and T. Masquelier. "Technical report: supervised training of convolutional spiking neural networks with PyTorch." In https://arxiv.org/abs/1911.10124

The github of their work can be found [here](https://github.com/romainzimmer/s2net).

The coding parts are adapted from the tutorial on surrogate gradient learning in spiking neural networks by [Friedemann Zenke](https://fzenke.net). This tutorial can be found [here](https://github.com/fzenke/spytorch) and the article related to it is referenced below:
>[2] Neftci, E.O., Mostafa, H., and Zenke, F. "Surrogate Gradient Learning in Spiking Neural Networks.", 2019. In https://arxiv.org/abs/1901.09948

Find all the bibliography in [Section 4](#sec4) and some useful ressources to go further at the end of each part.

1. [What are SNN?](#sec1)
    1. [Functioning and intro to SNNs](#sec1-1)
    2. [Integrate and Fire Neuron Models](#sec1-2)
    3. [Building a small network on Pytorch](#sec1-3)
2. [Training SNNs](#sec2)
    1. [Why usual methods do not work?](#sec2-1)
    2. [Supervised Learning with surrogate gradient](#sec2-2)
    3. [Other training methods](#sec2-3)
3. [Using SNNs for classification problems](#sec3) 
    1. [Classifying images](#sec3-1)
    2. [Audio recognition](#sec3-2)
    3. [Deep SNNs](#sec3-3)
    4. [Convolutional SNNs for speech recognition](#sec3-4)
4. [Bibliography](#sec4)

<div class="alert alert alert-info"><b> Important information:</b><br> This notebook can be executed locally on our own machine but some examples might take a while. The best would be to use Google Colab on these parts. You will see a warning cell before launching them to tell you to switch to Colab. </div>

Make sure you have all the requiered libraries installed by running the code below:

In [ ]:
!pip install numpy matplotlib seaborn torch

# 1. <a id="sec1"></a> What are SNN?

## <a id="sec1-1"></a> 1.1 Functioning and intro to SNNs

Artificial Neural Networks (ANNs) have tried for years to mimic the human brain. But they are characterized by a continuous-valued activation while biological neurons use discrete spikes to transmit information. To get closer to a neuromorphic architecture, one can use Spiking Neural Networks (SNNs). SNNs are considered the third generation of Neural Networks.

Unlike standard multi-layer perceptron networks, neurons in an SNN transmit information only when a membrane potential reaches a threshold. Then, the neuron fires, and generates a signal that travels to other neurons which, in turn, increase or decrease their potentials. The output of a spiking neuron, called a train spike, is described by the time sequence of spikes issued by it.

<img src="img/spiking_neuron.png" width="300px"></img><br><i>Figure from [3]: Spiking Neuron
    
Spike trains are propagated through synaptic connections. A synapse can be either excitatory, which increases the neuron’s membrane potential upon receiving input, or inhibitory, which decreases the neuron’s membrane potential. The strength of the adaptive synapses (weights) can be changed as a result of learning. The learning rule of an SNN is its most challenging component for developing multi-layer (deep) SNNs, because the non-differentiability of spike trains limits the popular backpropagation algorithm [4]. In this notebook we will see how to train them with surrogate gradients.
    
Two neurons $i$ and $j$ are connected by a "synapse" of weight $w_{ij}$.

<img src="img/wij.PNG" width="300px"></img><br><i>Figure from [5]: Pre-synaptic and Post-synaptic neurons
    
$i$ is called the pre-synaptic neuron and $j$ the post-synaptic one.

In addition to their brain-like structure, SNNs come with many advantages: they are hardware friendly, energy efficient if implemented on specialized neuromorphic hardware, require low-power, and collocate memory and processing [1]. Owing to the incorporation of temporal dimension, SNNs naturally lend themselves for processing of signals in time such as audio, video, speech, etc.

## <a id="sec1-2"></a> 1.2 Integrate and Fire Neuron Models

Now, let's talk about the of neurons used in SNNs. Mainly there are two: Leaky Integrate and Fire Neurons (LIF) and Non Leaky Integrate and Fire Neurons (NLIF).

For a LIF neuron $i$ in layer $l$, the subthreshold dynamics of its membrane is given by the following equation:

$$ \textrm{ (I) } \tau_\mathrm{mem} \frac{\mathrm{d}U_i^{(l)}}{\mathrm{d}t} = -(U_i^{(l)}-U_\mathrm{rest}) + RI_i^{(l)}$$

Where:
- $U_i(t)$ is the membrane potential at time $t$
- $U_{rest}$ is the resting membrane potential
- $\tau_{mem}$ is the membrane time constant
- $I_i$ is the current injected into the neuron
- $R$ is the restistance

When $U_i>\Delta_i$ (threshold), the neuron fires and $U_i$ decreases.<br>
$-(U_i-U_{rest})$ corresponds to the leak term and, as $U_i$ decreases, it tends toward $U_{rest}$.

We write for the spike train (ie. the sum of all spikes $C_i^l$ emitted by neuron $i$ in layer $l$):

$$S_i^{(l)}(t)=\sum_{k \in C_i^l} \delta(t-t_j^k)$$

- $\delta$ the Dirac delta function
- $t_i^k$ the associated firing times of the neuron

The LIF model is extremely popular because it captures the intuitive properties of external input accumulating charge across a leaky cell membrane with a clear threshold [3].

For a NLIF neuron, the equation is similar but without the leak term:

$$ \textrm{ (II) } \tau_{mem}\frac{dU_i}{dt}=RI_i $$

For the rest of the presentation, we will focus on LIF neurons.

Spikes travel down the axon and generate a postsynaptic currents in connected neurons:

$$\frac{\mathrm{d}I_i}{\mathrm{d}t}= -\frac{I_i(t)}{\tau_\mathrm{syn}} + \sum_j W_{ij} S_j^{(0)}(t) + \sum_j V_{ij} S_j^{(1)}(t)$$

- $W_{ij}$ the synaptic weight matrices (feed-forward)
- $V_{ij}$ recurrent connections.
- $\tau_\mathrm{syn}$ the synaptic decay time constant.

We can now express the aboves equations on a whole network in discrete time. We will take $U_\mathrm{rest}=0$, $R=1$, and $\Delta=1$. Our arguments remain unaffected by this choice, and all results can always be re-scaled back to physical units.

We can set:

$$S_i^{(l)}(t)=\Theta(U_i^{(l)}(t)-\Delta)$$

where $\Theta$ denotes the Heaviside step function.

Assuming a small simulation time step of $\Delta_t>0$ we can approximate the synaptic dynamics by:

$$I_i^{(l)}(t+1) = \alpha I_i^{(l)}(t) + \sum_j W_{ij} S_j^{(l-1)}(t) +\sum_j V_{ij} S_j^{(l)}(t)$$

With:
- $\alpha=\exp\left(-\frac{\Delta_t}{\tau_\mathrm{syn}} \right)$ Further, the membrane dynamics can be written as:

$$U_i^{(l)}(t+1) = \underbrace{\beta U_i^{(l)}(t)}_{\mathrm{leak}} + \underbrace{I_i^{(l)}(t)}_{\mathrm{input}} -\underbrace{S_i^{(l)}(t)}_{\mathrm{reset}}$$

- $S_i(t) = \Theta(U_i(t)-1)$ the output
- $\beta=\exp\left(-\frac{\Delta_t}{\tau_\mathrm{mem}}\right)$

The terms on the right-hand-side of the equation are responsible individually for i) leak, ii) synaptic input, and iii) the spike reset.

## <a id="sec1-3"></a> 1.3 Building a small network with Pytorch

We will first construct a small network with only one hidden layer, no recurrent connection and that should classified a set of different sparse input spike trains into two categories.

Just a reminder: this is adapted from the first tutorial of "Training a spiking neural network with surrogate gradients" by Friedmann Zenke. The original code can be found [here](https://github.com/fzenke/spytorch/blob/master/notebooks/SpyTorchTutorial1.ipynb).

First we define our variables and import what we need.

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

import torch
import torch.nn as nn
import torchvision

dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")     
else:
    device = torch.device("cpu")

In [ ]:
nb_inputs  = 100
nb_hidden  = 4
nb_outputs = 2

In [ ]:
time_step = 1e-3
nb_steps  = 200
batch_size = 256

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

Then we generate our data and plot it to see how it looks:

In [ ]:
freq = 5 # Hz
prob = freq*time_step
mask = torch.rand((batch_size,nb_steps,nb_inputs), device=device, dtype=dtype)
x_data = torch.zeros((batch_size,nb_steps,nb_inputs), device=device, dtype=dtype, requires_grad=False)
x_data[mask<prob] = 1.0

In [ ]:
data_id = 0
plt.imshow(x_data[data_id].cpu().t(), cmap=plt.cm.gray_r, aspect="auto")
plt.xlabel("Time (ms)")
plt.ylabel("Unit")
sns.despine()

Next, we assign a random label of 0 or 1 to each of our input patterns. Our network's task will be to differentiate these patterns.

In [ ]:
y_data = torch.tensor(1*(np.random.rand(batch_size)<0.5), device=device)

Now we set up our weight matrices, which connect the input and the hidden layer, as well as the matrix connecting the hidden layer with the output layer. We initialize these weights randomly from a normal distribution.

In [ ]:
weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

w1 = torch.empty((nb_inputs, nb_hidden),  device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w1, mean=0.0, std=weight_scale/np.sqrt(nb_inputs))

w2 = torch.empty((nb_hidden, nb_outputs), device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w2, mean=0.0, std=weight_scale/np.sqrt(nb_hidden))

To implement our spiking neuron we need to multiply all input spikes with the weight matrix for each time step in each input example in the batch.

In [ ]:
h1 = torch.einsum("abc,cd->abd", (x_data, w1))

In discrete-time, we can formulate our spiking nonlinearity as a Heaviside step function:

In [ ]:
def spike_fn(x):
    out = torch.zeros_like(x)
    out[x > 0] = 1.0
    return out

For each trial, we initialize the synaptic currents and membrane potentials at zero. Then, we implement a loop that simulates our neuron models over time for "nb_steps":

In [ ]:
syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)

# Here we define two lists which we use to record the membrane potentials and output spikes
mem_rec = [mem]
spk_rec = [mem]

# Here we loop over time
for t in range(nb_steps):
    mthr = mem-1.0
    out = spike_fn(mthr)
    rst = torch.zeros_like(mem)
    c = (mthr > 0)
    rst[c] = torch.ones_like(mem)[c]
    
    new_syn = alpha*syn +h1[:,t]
    new_mem = beta*mem +syn -rst
    
    mem = new_mem
    syn = new_syn
    
    mem_rec.append(mem)
    spk_rec.append(out)

# Now we merge the recorded membrane potentials into a single tensor
mem_rec = torch.stack(mem_rec,dim=1)
spk_rec = torch.stack(spk_rec,dim=1)

Now we can plot multiple trials at once:

In [ ]:
def plot_voltage_traces(mem, spk=None, dim=(3,5), spike_height=5):
    gs=GridSpec(*dim)
    if spk is not None:
        dat = (mem+spike_height*spk).detach().cpu().numpy()
    else:
        dat = mem.detach().cpu().numpy()
    for i in range(np.prod(dim)):
        if i==0: a0=ax=plt.subplot(gs[i])
        else: ax=plt.subplot(gs[i],sharey=a0)
        ax.plot(dat[i])
        ax.axis("off")

In [ ]:
fig=plt.figure(dpi=100)
plot_voltage_traces(mem_rec, spk_rec)

Now that we have an input layer, a spiking layer (hidden), we need a readout layer to use our network as a classifier. Here we will defined an objective function on the membrane potential of the output neurons but there are many other possibilities.

All the previous code, plus the readout layer, was rewritten in one single function:

In [ ]:
def run_snn(inputs):
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)

    mem_rec = [mem]
    spk_rec = [mem]

    # Compute hidden layer activity
    for t in range(nb_steps):
        mthr = mem-1.0
        out = spike_fn(mthr)
        rst = torch.zeros_like(mem)
        c   = (mthr > 0)
        rst[c] = torch.ones_like(mem)[c]

        new_syn = alpha*syn +h1[:,t]
        new_mem = beta*mem +syn -rst

        mem = new_mem
        syn = new_syn

        mem_rec.append(mem)
        spk_rec.append(out)

    mem_rec = torch.stack(mem_rec,dim=1)
    spk_rec = torch.stack(spk_rec,dim=1)

    # Readout layer
    h2= torch.einsum("abc,cd->abd", (spk_rec, w2))
    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]
    for t in range(nb_steps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = [mem_rec, spk_rec]
    return out_rec, other_recs

We can now plot the output layer "membrane potentials":

In [ ]:
out_rec,other_recs = run_snn(x_data)

In [ ]:
fig=plt.figure(dpi=100)
plot_voltage_traces(out_rec)

We take the maximum voltage over time of each output unit:
$$\hat U^\mathrm{out}_i=\max_t U^\mathrm{out}_i(t)$$
and then use this vector as input for a standard softmax function to compute the classification accuracy, with a negative log-likelihood loss. 

In [ ]:
def print_classification_accuracy():
    """ Dirty little helper function to compute classification accuracy. """
    output,_ = run_snn(x_data)
    m,_= torch.max(output,1) # max over time
    _,am=torch.max(m,1) # argmax over output units
    acc = np.mean((y_data==am).detach().cpu().numpy()) # compare to labels
    print("Accuracy %.3f"%acc)
    
print_classification_accuracy()

It is about 50% because the network is not trained so the probability to classified an input as a 0 or a 1 is 0.5.

<div class="alert alert alert-success"><b> Part I in short:</b><br>
SNNs are the third generation of Neural Networks, based on sparse binary activation, they are biologically realistic and efficient in the processing of signals in time. But they can be difficult to train due to the non-differentiability of their inputs.<br>  
There are two types of neurons that can be used in SNNs: LIF and NLIF neurons. They are both described by differential equations, with or without leak term. We can thus build a network with those neurons by expressing the synaptic currents $I_i^{(l)}$ and the membrane potentials $U_i^{(l)}$ in discrete time. Pytorch can be used to build such networks.<br>
But what about training them?
</div>

### Going Further

A great place to learn more about SNNs is [wikipedia](https://en.wikipedia.org/wiki/Spiking_neural_network). But if you want something more details, here is an article that summarize it well: as early as 2003, J. Vreeken was presenting the concept of SNNs with all the technical terms you need to know.
> [6] J. Vreeken. "Spiking neural networks, an introduction.", 2003. http://www.ai.jonad.eu/materialy/download/sieci_neuronowe/2003-008.pdf

# 2. <a id="sec2"></a> Training SNNs

## <a id="sec2-1"></a> 2.1 Why usual methods do not work?

We have successfully created a small spiking neural network, but it do not classify correctly our data just yet as it is not trained. It is now time to adjust the network's weights using Supervised Learning.

But, haven't we say earlier that such networks were hard to train? Yes! It is one of the numerous open research questions around SNNs. But why the methods we already know cannot work this time?

Well to illustrate this, let's try to perform gradient descent with the true gradient:

<div class="alert alert alert-warning"><b> Warning:</b> This can take a few minutes, don't panic!<\div>

In [ ]:
params = [w1,w2] # The paramters we want to optimize
optimizer = torch.optim.Adam(params, lr=2e-3, betas=(0.9,0.999)) # The optimizer we are going to use

log_softmax_fn = nn.LogSoftmax(dim=1) # The log softmax function across output units
loss_fn = nn.NLLLoss() # The negative log likelihood loss function

# The optimization loop
loss_hist = []
for e in range(1000):
    # run the network and get output
    output,_ = run_snn(x_data) 
    # compute the loss
    m,_=torch.max(output,1)
    log_p_y = log_softmax_fn(m) 
    loss_val = loss_fn(log_p_y, y_data)

    # update the weights
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()
    
    # store loss value
    loss_hist.append(loss_val.item())
    
loss_hist_true_grad = loss_hist # store for later use

In [ ]:
plt.plot(loss_hist)
plt.xlabel("Epoch")
plt.ylabel("Loss")
sns.despine()

In [ ]:
print_classification_accuracy()

As you can see, loss decreases over iterations and converges towards a steady state. Yet it is far from a state of the art accuracy.

The reason is that the nonlinearity of the hidden units have zero derivatives everywhere except at threshold crossings, where they become infinite.

Thus, weight updates in the hidden layer vanish and the weights remain unmodified: no learning happens in the hidden layer. The loss decreased during optimization because the output layer weights could still change and allow for some (very little) improvement.

In [ ]:
output,other_recordings = run_snn(x_data)
mem_rec, spk_rec = other_recordings

In [ ]:
fig=plt.figure(dpi=100)
plot_voltage_traces(mem_rec, spk_rec)

In [ ]:
fig=plt.figure(dpi=100)
plot_voltage_traces(output)

Mathematicaly, we can write the following formula, obtained from the chain rule, that occurs in all variants of backpropagation: [4]

$$ \delta_j^{\mu} = g^{′}(a_j^{\mu}) \sum_k w_{kj} \delta_k^{\mu} $$

With:
- input pattern $\mu$
- $\delta_j^{\mu}$ and $\delta_k^{\mu}$ the partial derivative of the cost function with respect to the net input to some arbitrary unit $j$ or $k$
- Unit $j$ projects direct feedforward connections to the set of units indexed by $k$
- $g(·)$ is the activation function applied to the net input of unit $j$
- Net input $a_j^{\mu}$
- $w_{kj}$ are the feedforward weights projecting from unit $j$ to the set of units indexed by $k$

But this formula contains two issues for the backpropagation procedure. First, the expression $g′(·)$ requires $g(·)$ with respect to $w_{kj}$. Since $g(·)$ applies to a spiking neuron, it is likely represented by a sum of Dirac delta functions, which means the derivative does not exist. The second applies to both spiking and non-spiking networks and termed the "weight transport" problem. The expression above uses the feedforward weights $w_{kj}$ in a feedback fashion. This means that matching symmetric feedback weights must exist and project accurately to the correct neurons (point-to-point feedback) in order for the equation to be usable.

We will here focus on the first issue.

So our problem is that the partial derivative of the step function we use has a vanishing derivative everywhere (except at zero where it becomes infinite). Most conventional neural networks avoid this problem by choosing sigmoidal or tanh units. But what if we if we want to stick to our binary nonlinearity?

There have been several approaches to tackle this problem. Here we use one such strategy which has been applied successfully to spiking neural networks: a surrogate gradient approach.

## <a id="sec2-2"></a> 2.2 Supervized Learning with Surrogate Gradient

The idea behind a surrogate gradient is simple. Instead of changing the nonlinearity itself, we only change the gradient. Thus we use a different "surrogate" gradient to optimize parameters that would otherwise have a vanishing gradient. We use the partial derivative of a function which to some extent approximates the stepfunction $\Theta(x)$.

<img src="img/surrgrad.png" width="450">

Here, we will use the partial derivative of a fast sigmoid function $\sigma(x)$. 
While $\Theta$ is invariant to multiplicative rescaling, $\sigma$ isn't. Thus we have to introduce a scale parameter.

In [ ]:
class SurrGradSpike(torch.autograd.Function):
    """
    Here we implement our spiking nonlinearity which also implements 
    the surrogate gradient. By subclassing torch.autograd.Function, 
    we will be able to use all of PyTorch's autograd functionality.
    Here we use the normalized negative part of a fast sigmoid 
    as this was done in Zenke & Ganguli (2018).
    """
    
    scale = 100.0 # controls steepness of surrogate gradient

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we compute a step function of the input Tensor
        and return it. ctx is a context object that we use to stash information which 
        we need to later backpropagate our error signals. To achieve this we use the 
        ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        out = torch.zeros_like(input)
        out[input > 0] = 1.0
        return out

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor we need to compute the 
        surrogate gradient of the loss with respect to the input. 
        Here we use the normalized negative part of a fast sigmoid 
        as this was done in Zenke & Ganguli (2018).
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad = grad_input/(SurrGradSpike.scale*torch.abs(input)+1.0)**2
        return grad
    
# here we overwrite our naive spike function by the "SurrGradSpike" nonlinearity which implements a surrogate gradient
spike_fn  = SurrGradSpike.apply

In [ ]:
# The following lines will reinitialize the weights
torch.nn.init.normal_(w1, mean=0.0, std=weight_scale/np.sqrt(nb_inputs))
torch.nn.init.normal_(w2, mean=0.0, std=weight_scale/np.sqrt(nb_hidden))

<div class="alert alert alert-warning"><b> Warning:</b> Same as before. Go get a coffee, you need it!<\div>

In [ ]:
params = [w1,w2]
optimizer = torch.optim.Adam(params, lr=2e-3, betas=(0.9,0.999))

log_softmax_fn = nn.LogSoftmax(dim=1)
loss_fn = nn.NLLLoss()

loss_hist = []
for e in range(1000):
    output,_ = run_snn(x_data)
    m,_=torch.max(output,1)
    log_p_y = log_softmax_fn(m)
    loss_val = loss_fn(log_p_y, y_data)

    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()
    loss_hist.append(loss_val.item())

In [ ]:
plt.figure(figsize=(3.3,2),dpi=150)
plt.plot(loss_hist_true_grad, label="True gradient")
plt.plot(loss_hist, label="Surrogate gradient")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
sns.despine()

Tadaaa

In [ ]:
output,other_recordings = run_snn(x_data)
mem_rec, spk_rec = other_recordings

In [ ]:
fig=plt.figure(dpi=100)
plot_voltage_traces(mem_rec, spk_rec)

In [ ]:
fig=plt.figure(dpi=100)
plot_voltage_traces(output)

In [ ]:
output,_ = run_snn(x_data)
m,_=torch.max(output,1)

# Compute training accuracy
_,am=torch.max(m,1)
acc = np.mean((y_data==am).detach().cpu().numpy())
print("Accuracy %f"%acc)

The accuracy is great, but have we totally solved our problem?

Yes and no.

One must be aware that these solutions are not bio-plausible. However, they are still useful from both engineering and scientific standpoints.

## <a id="sec2-3"></a> 2.3 Other training methods

The surrogate gradient method is not the only way to solve our issues.

There is a whole part of litterature focusing on spike-timing-dependent plasticity (STDP). STDP is an umbrella term reuniting several methods to train SNNs, often unsupervized. Its key feature is that the weight (synaptic efficacy) connecting a pre- and post-synaptic neuron is adjusted according to their relative spike times. The information used to perform the weight adjustment is both local to the synapse and local in time. The most common form of biological STDP consists in saying that if a presynaptic neuron fires briefly (≈10 ms) before the postsynaptic neuron, the weight connecting them is strengthened. If the presynaptic neuron fires briefly after the postsynaptic neuron, then the causal relationship between the temporal events is spurious and the weight is weakened. Strengthening is called long-term potentiation (LTP) and weakening is called long-term depression (LTD) [4][5].

If we want to stick to supervized learning, surrogate gradient using substitute or approximate derivatives as we saw works well but is not very biorealistic but it is possible to make it more plausible by using an activity regularizer, which encourages solutions with sparse spiking.

For the issue of weight transport, it was shown that for some tasks, backpropagation could still perform well if random feedback weights were used or with mirrored STDP learning by building an STDP trained auto-encoder with
matched-weight, point-to-point connections across layers [4]. But these approaches also have their drawbacks. This is why the training method for SNNs is still an open research question.

A summary of the literature focusing on traing SNNs shows us that the possible approaches are numerous:

<img src="img/table-summary.PNG" width="900"><br><i>
Figure from [4]: Summary of recent deep learning models developed in SNN platforms and their accuracy on different datasets.

<div class="alert alert alert-success"><b> Part II in short:</b><br>
Training SNNs can be tricky. If you perform a gradient descent on your network the hidden spiking layers will not be trained. This is due to the vanishing derivative of the step function we use.<br>
One of the solution is simply to use a "surrogate" gradient: an approximation of the gradient using a rescaled sigmoid function.<br>
This method is not the only one to train SNNs and how to improve this training to be both efficient and brain-plausible is still an open research question.
</div>

#### Going Further

For a broad vision on the different methods that exist today to train SNNs one can look at the following article.
>[4] Amirhossein Tavanaei, Masoud Ghodrati, Saeed Reza Kheradpisheh, Timothée Masquelier, Anthony Maida. "Deep learning in spiking neural networks." Neural Networks, Volume 111, 2019, Pages 47-63. ISSN 0893-6080, https://doi.org/10.1016/j.neunet.2018.12.002

For a course on unsupervised training on SNNs with STDP (way behind the scope of this notebook), look at this one.
>[5] Ted Pavlic. "IEE/CSE 598: Lecture 7G (2020-04-15) - Intro. to Spiking Neural Networks and Neuromorphic Computing." On https://www.youtube.com/watch?v=ICw2_49dSNw&ab_channel=TedPavlic

To see how to train an image classifier with a surrogate gradient but with an activity regularizer to encourage solutions with sparse spiking, look at the [third tutorial](https://github.com/fzenke/spytorch/blob/master/notebooks/SpyTorchTutorial3.ipynb) of F. Zenke.

# 3. <a id="sec3"></a> Using SNNs for classification problems

As far, we have only seen a very simple network. But SNNs can be used to classified much larger datasets.

## <a id="sec3-1"></a> 3.1 Classifying images

We can easily use what we have learned to classify images from the [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist) for instance.

This is adapted from the second tutorial of F. Zenke. Original code [here](https://github.com/fzenke/spytorch/blob/master/notebooks/SpyTorchTutorial2.ipynb).

In [ ]:
# The coarse network structure is dicated by the Fashion MNIST dataset. 
nb_inputs  = 28*28
nb_hidden  = 100
nb_outputs = 10

time_step = 1e-3
nb_steps  = 100

batch_size = 256

In [ ]:
# Here we load the Dataset
root = os.path.expanduser("~/data/datasets/torch/fashion-mnist")
train_dataset = torchvision.datasets.FashionMNIST(root, train=True, transform=None, target_transform=None, download=True)
test_dataset = torchvision.datasets.FashionMNIST(root, train=False, transform=None, target_transform=None, download=True)

In [ ]:
# Standardize data
# x_train = torch.tensor(train_dataset.train_data, device=device, dtype=dtype)
x_train = np.array(train_dataset.data, dtype=np.float)
x_train = x_train.reshape(x_train.shape[0],-1)/255
# x_test = torch.tensor(test_dataset.test_data, device=device, dtype=dtype)
x_test = np.array(test_dataset.data, dtype=np.float)
x_test = x_test.reshape(x_test.shape[0],-1)/255

# y_train = torch.tensor(train_dataset.train_labels, device=device, dtype=dtype)
# y_test  = torch.tensor(test_dataset.test_labels, device=device, dtype=dtype)
y_train = np.array(train_dataset.targets, dtype=np.int)
y_test  = np.array(test_dataset.targets, dtype=np.int)

In [ ]:
# Here we plot one of the raw data points as an example
data_id = 1
plt.imshow(x_train[data_id].reshape(28,28), cmap=plt.cm.gray_r)
plt.axis("off")

Since we are working with spiking neural networks, we ideally want to use a temporal code to make use of spike timing. To that end, we will use a spike latency code to feed spikes to our network.

In [ ]:
def current2firing_time(x, tau=20, thr=0.2, tmax=1.0, epsilon=1e-7):
    """ Computes first firing time latency for a current input x assuming the charge time of a current based LIF neuron.

    Args:
    x -- The "current" values

    Keyword args:
    tau -- The membrane time constant of the LIF neuron to be charged
    thr -- The firing threshold value 
    tmax -- The maximum time returned 
    epsilon -- A generic (small) epsilon > 0

    Returns:
    Time to first spike for each "current" x
    """
    idx = x<thr
    x = np.clip(x,thr+epsilon,1e9)
    T = tau*np.log(x/(x-thr))
    T[idx] = tmax
    return T
 

def sparse_data_generator(X, y, batch_size, nb_steps, nb_units, shuffle=True ):
    """ This generator takes datasets in analog format and generates spiking network input as sparse tensors. 

    Args:
        X: The data ( sample x event x 2 ) the last dim holds (time,neuron) tuples
        y: The labels
    """

    labels_ = np.array(y,dtype=np.int)
    number_of_batches = len(X)//batch_size
    sample_index = np.arange(len(X))

    # compute discrete firing times
    tau_eff = 20e-3/time_step
    firing_times = np.array(current2firing_time(X, tau=tau_eff, tmax=nb_steps), dtype=np.int)
    unit_numbers = np.arange(nb_units)

    if shuffle:
        np.random.shuffle(sample_index)

    total_batch_count = 0
    counter = 0
    while counter<number_of_batches:
        batch_index = sample_index[batch_size*counter:batch_size*(counter+1)]

        coo = [ [] for i in range(3) ]
        for bc,idx in enumerate(batch_index):
            c = firing_times[idx]<nb_steps
            times, units = firing_times[idx][c], unit_numbers[c]

            batch = [bc for _ in range(len(times))]
            coo[0].extend(batch)
            coo[1].extend(times)
            coo[2].extend(units)

        i = torch.LongTensor(coo).to(device)
        v = torch.FloatTensor(np.ones(len(coo[0]))).to(device)
    
        X_batch = torch.sparse.FloatTensor(i, v, torch.Size([batch_size,nb_steps,nb_units])).to(device)
        y_batch = torch.tensor(labels_[batch_index],device=device)

        yield X_batch.to(device=device), y_batch.to(device=device)

        counter += 1

In [ ]:
tau_mem = 10e-3
tau_syn = 5e-3

alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

In [ ]:
weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

w1 = torch.empty((nb_inputs, nb_hidden),  device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w1, mean=0.0, std=weight_scale/np.sqrt(nb_inputs))

w2 = torch.empty((nb_hidden, nb_outputs), device=device, dtype=dtype, requires_grad=True)
torch.nn.init.normal_(w2, mean=0.0, std=weight_scale/np.sqrt(nb_hidden))

print('init done')

In [ ]:
class SurrGradSpike(torch.autograd.Function):
    """
    Here we implement our spiking nonlinearity which also implements 
    the surrogate gradient. By subclassing torch.autograd.Function, 
    we will be able to use all of PyTorch's autograd functionality.
    Here we use the normalized negative part of a fast sigmoid 
    as this was done in Zenke & Ganguli (2018).
    """
    
    scale = 100.0 # controls steepness of surrogate gradient

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we compute a step function of the input Tensor
        and return it. ctx is a context object that we use to stash information which 
        we need to later backpropagate our error signals. To achieve this we use the 
        ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        out = torch.zeros_like(input)
        out[input > 0] = 1.0
        return out

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor we need to compute the 
        surrogate gradient of the loss with respect to the input. 
        Here we use the normalized negative part of a fast sigmoid 
        as this was done in Zenke & Ganguli (2018).
        """
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad = grad_input/(SurrGradSpike.scale*torch.abs(input)+1.0)**2
        return grad
    
# here we overwrite our naive spike function by the "SurrGradSpike" nonlinearity which implements a surrogate gradient
spike_fn  = SurrGradSpike.apply

In [ ]:
def run_snn(inputs):
    h1 = torch.einsum("abc,cd->abd", (inputs, w1))
    syn = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)
    mem = torch.zeros((batch_size,nb_hidden), device=device, dtype=dtype)

    mem_rec = [mem]
    spk_rec = [mem]

    # Compute hidden layer activity
    for t in range(nb_steps):
        mthr = mem-1.0
        out = spike_fn(mthr)
        rst = torch.zeros_like(mem)
        c   = (mthr > 0)
        rst[c] = torch.ones_like(mem)[c]

        new_syn = alpha*syn +h1[:,t]
        new_mem = beta*mem +syn -rst

        mem = new_mem
        syn = new_syn

        mem_rec.append(mem)
        spk_rec.append(out)

    mem_rec = torch.stack(mem_rec,dim=1)
    spk_rec = torch.stack(spk_rec,dim=1)

    # Readout layer
    h2= torch.einsum("abc,cd->abd", (spk_rec, w2))
    flt = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out = torch.zeros((batch_size,nb_outputs), device=device, dtype=dtype)
    out_rec = [out]
    for t in range(nb_steps):
        new_flt = alpha*flt +h2[:,t]
        new_out = beta*out +flt

        flt = new_flt
        out = new_out

        out_rec.append(out)

    out_rec = torch.stack(out_rec,dim=1)
    other_recs = [mem_rec, spk_rec]
    return out_rec, other_recs

In [ ]:
def train(x_data, y_data, lr=2e-3, nb_epochs=10):
    params = [w1,w2]
    optimizer = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))

    log_softmax_fn = nn.LogSoftmax(dim=1)
    loss_fn = nn.NLLLoss()
    
    loss_hist = []
    for e in range(nb_epochs):
        local_loss = []
        for x_local, y_local in sparse_data_generator(x_data, y_data, batch_size, nb_steps, nb_inputs):
            output,_ = run_snn(x_local.to_dense())
            m,_=torch.max(output,1)
            log_p_y = log_softmax_fn(m)
            loss_val = loss_fn(log_p_y, y_local)

            optimizer.zero_grad()
            loss_val.backward()
            optimizer.step()
            local_loss.append(loss_val.item())
        mean_loss = np.mean(local_loss)
        print("Epoch %i: loss=%.5f"%(e+1,mean_loss))
        loss_hist.append(mean_loss)
        
    return loss_hist
        
        
def compute_classification_accuracy(x_data, y_data):
    """ Computes classification accuracy on supplied data in batches. """
    accs = []
    for x_local, y_local in sparse_data_generator(x_data, y_data, batch_size, nb_steps, nb_inputs, shuffle=False):
        output,_ = run_snn(x_local.to_dense())
        m,_= torch.max(output,1) # max over time
        _,am=torch.max(m,1)      # argmax over output units
        tmp = np.mean((y_local==am).detach().cpu().numpy()) # compare to labels
        accs.append(tmp)
    return np.mean(accs)

<div class="alert alert alert-warning"><b> Warning:</b> By now, you should know the drill... But this one is especially long. If you are not on Colab or if you do not have a gpu, skip it. Even on Colab this could take like 15min, so be prepared! (or skip it, no big deal) <\div>

If you really want to wait for the result, here is a little game you probably know already, to pass the time: [Grrr!!!](https://trex-runner.com/)

In [ ]:
loss_hist = train(x_train, y_train, lr=2e-4, nb_epochs=30)

In [ ]:
plt.figure(figsize=(3.3,2),dpi=150)
plt.plot(loss_hist)
plt.xlabel("Epoch")
plt.ylabel("Loss")
sns.despine()

In [ ]:
print("Training accuracy: %.3f"%(compute_classification_accuracy(x_train,y_train)))
print("Test accuracy: %.3f"%(compute_classification_accuracy(x_test,y_test)))

## <a id="sec3-2"></a> 3.2 Audio recognition

To train a network to recognize audio commands, the inputs have to be spikes trains. Several methods are possible. In the previous section, we used a simple time-to-first-spike code. Some have chosen a sophisticated cochlear model to generate the spike data corresponding to audio recording [7].

Our main article [1] chose another strategy.

### Event-based sampling

Most of the times, the input signal is real-valued and as to be encoded as spike trains. To do that we can use different event-based sampling strategies. Here we will use the Send-on-Delta (SoD) strategy. The sampling is triggered when a significant change ($\Delta$) is detected in signal $x$:

$$ \textrm{ (3) } t_k=\inf_{t \geq t_{k-1}} \{ t, \lvert x(t) - x(t_{k-1} \rvert \geq \Delta \} $$

<img src="img/graph_delta.PNG" width="500px"></img><br> <i>Figure from [1]: SoD sampling strategy<br>
Blue dots (resp. red) represent sampling due to significant increase (resp. decrease)

For this strategy we use two NLIF neurons:

$$I(t)=w \dot{x}(t) $$
$$U(t_k^{+})=0$$

With:
- $t_k^{+}$ (resp. $t_k^{-}$) the time right after (resp. before) the $k^{th}$ spike emitted
- w the scaling factor

Which gives us:

$$ \textrm{ (4) } U(t)=w(x(t)-x(t_k))$$

With a threshold $w^2$, $(3)$ and $(4)$ give that the next spike is emitted at $t_{k+1}$ such as:

$$t_{k+1}= \inf_{t \geq t_k} \{ t, U(t) \geq w^2 \} $$

We use two neurons : one ON ($w_{ON} > 0$) and one OFF ($w_{OFF} < 0$). The potential reset when any of them fires: $U_i(t_k^{+})=0, \forall k$

If ON fires at $t_{k+1}$, we have:

$$U_{ON}(t_{k+1}^{-})=w_{ON}(x(t_{k+1})-x(t_{k}))=w_{ON}^2$$
$$U_{OFF}(t_{k+1}^{-})=w_{OFF}(x(t_{k+1})-x(t_{k}))=w_{OFF}w_{ON}$$

The weight of their lateral connection must then be $-w_{OFF}w_{ON}$ to reset the potential of both neurons when any of them fires.

<img src="img/ONOFF.PNG" width="500px"></img><br><i>Figure from [1]: Architecture for SoD encoding with two IF neurons

We used 2 neurons to implement an SoD strategy on a 1D signal. For a mD signal, we need 2*m neurons to detect changes in each dimension independently. But we can also detect changes in projections of these dimentions along $w_i$ the preferred dimension of neuron $i$. Then:

$ U_i(t)=\langle w_i x(t)-x(t_k) \rangle $ with threshold $ \|w_i\|^2 $ and lateral connection between neurons $i$ and $j$ is $- \langle w_j,w_i \rangle $.

If we were to do the same with LIF neurons, only abrupt changes will be detected.

## <a id="sec3-3"></a> 3.3 Deep SNNs

Deep learning uses an architecture with many layers of trainable parameters and has demonstrated outstanding performance with the second generation of artificial neural networks. With SNNs, progress are being make to train these huge networks efficiently. As in regular DNNs, spiking layers can be fully connected, convolutional, recurrent...

As a reminder, when we approximate (I) by linear recurrent equations in discrete time (cf [1] Appendix A), the neuron $i$ dynamic at step $n$ is fully discribed by the following equations:

$$U_i[n]= \beta (U_i[n-1]-R_i[n]+(1- \beta) I_i[n]$$
$$I_i[n]= \sum_j W_{ij} S_j^{in}[n]$$
$$R_i[n]=(W . W^T . S^{out}[n])_i$$
$$S_i^{out}[n]= \Theta (U_i[n] - \Delta_i)$$
$$ \Delta_i= \|W_i \|^2 $$

With:
- $\beta= \exp{(-\frac{\Delta t }{\tau_{mem}})}$ and $\alpha= \exp{(-\frac{\Delta t }{\tau_{syn}})}$
- $ \Theta$ the Heaviside step function: can be approximated by a sigmoid function.

LIF neurons can thus be modeled as a RNN cells with at step n a state $(U[n],I[n])$ and an output $S[n]$.

The question is how a spiking CNN for instance can be trained while incorporating traditional CNNs properties. In the case of vision, the first layer of convolution is interpreted as extracting primary visual features.

In a fully connected spiking layer, the input at each step is a weighted sum of the input spikes emitted by the previous layer at this moment.

For the convolution spiking layer, the input current at each time step is given by a nD convolution between a kernel and the input spike train.

In audio recognition, event-sampling strategies can be used like shown in [Section 3-2](#sec3-2). But before that, audio signals need to be prepocessed.

For the recurrent model, the output is fed back to the neurons.

The last layer, if we want our network to be a classifier, should be a readout layer using a softmax function.

We can even penalized the number of spikes by applying a L1 or L2 loss on the total number of spikes emitted by each layer.

## <a id="sec3-4"></a> 3.4 Convolutional SNNs for speech recognition

If you want to see a great application of surrogate gradient learning on Convolutional SNNs, you can check the Speech Commands recognition task by [Romain Zimmer](https://github.com/romainzimmer) and his team.
>[1] R. Zimmer, T. Pellegrini2 , S. F. Singh, and T. Masquelier. "Technical report: supervised training of convolutional spiking neural networks with PyTorch." In https://arxiv.org/abs/1911.10124

The code can be found [here](https://github.com/romainzimmer/s2net). Unfortunately, the dataset was too large and the training too long to be executed for this class, even on Colab, but you can do it during your spare time if you want! 

To explain briefly the aim of this task:

The dataset contains audio recording of 30 different words. The aim is to discriminate 10 of them, the silences and the unknown commands (or other commands). After preprocessing the data, R. Zimmer uses an architecture composed of 3 convolutional layers, a readout layer fully connected. For the training he uses the Rectified Adam optimizer with a learning rate of $10^{-3}$, 30 epochs, 1 warm up and a weighted decay of $10^{-5}$. On each layer $l \in  [\![1;L]\!]$ , he applies a $L_r$ regularization loss with a coefficient $0.1 \ast \frac{l}{L}$.

The results show an accuracy of 93% against 97% on average with standard DL methods.

### Conclusion

The developement of SNNs are quite recent but as for now they do not necessarly give better results than usual DL networks. The goal is to get an accuracy close to standard ANNs models.<br>
Yet, training SNNs is one of the greatest challenges in the field today, but techniques have emerged to adapt backpropagation and gradient descent to those networks.<br>
Work on training recurrent SNNs appears to be in its infancy. But the emergence of energy-efficient, neuromorphic computing hardware has given the field a great boost.

<div class="alert alert alert-success"><b> Part III in short:</b><br>
With a simple spike latency code it is possible to classify images. But we can use other methods to encode images as spike trains in a more brain-plausible way.<br>
For audio recognition, it is possible to use two Integrate and Fire neurons to implement an event-based sampling strategies. We can even generalized that to a mD signal.<br>
In the end, we can build deep spiking neural netwoks, using spiking neurons in layers we already know about (convolutional, recurrent...). But the results are less accurate than with regular DNNs. Yet SNNs are promising to meet the next challenges in Machine Learning.
</div>

### Going Further

If you want to see a classification of spoken digits whose spikes were generated thanks to a cochlear model, see the [fourth tutorial](https://github.com/fzenke/spytorch/blob/master/notebooks/SpyTorchTutorial4.ipynb) by F. Zenke.

To know more about this model, check:
>[7] Cramer, B., Stradmann, Y., Schemmel, J., and Zenke, F. "The Heidelberg spiking datasets for the systematic evaluation of spiking neural networks.", 2019. In https://arxiv.org/abs/1910.07407

On the different architectures possible for Deep SNNs, see the part "Deep Learning in SNNs" in:
>[4] Amirhossein Tavanaei, Masoud Ghodrati, Saeed Reza Kheradpisheh, Timothée Masquelier, Anthony Maida. "Deep learning in spiking neural networks." Neural Networks, Volume 111, 2019, Pages 47-63. ISSN 0893-6080, https://doi.org/10.1016/j.neunet.2018.12.002

On the challenges we still have with SNNs, see:
>[8] André Grüning and Sander M. Bohte. "Spiking Neural Networks: Principles and Challenges." European Symposium on Artificial Neural Networks, Computational Intelligence and Machine Learning, 2014. In https://homepages.cwi.nl/~sbohte/publication/es2014-13Gruning.pdf

# 4. <a id="sec4"></a> Bibliography

[1] R. Zimmer, T. Pellegrini , S. F. Singh, and T. Masquelier. "Technical report: supervised training of convolutional spiking neural networks with PyTorch." In https://arxiv.org/abs/1911.10124

[2] Neftci, E.O., Mostafa, H., and Zenke, F. "Surrogate Gradient Learning in Spiking Neural Networks.", 2019. In https://arxiv.org/abs/1901.09948

[3] Anwani, Navin and B. Rajendran. "Training Multilayer Spiking Neural Networks using NormAD based Spatio-Temporal Error Backpropagation.", 2020 In https://arxiv.org/abs/1811.10678

[4] Amirhossein Tavanaei, Masoud Ghodrati, Saeed Reza Kheradpisheh, Timothée Masquelier, Anthony Maida. "Deep learning in spiking neural networks." Neural Networks, Volume 111, 2019, Pages 47-63. ISSN 0893-6080, https://doi.org/10.1016/j.neunet.2018.12.002

[5] Ted Pavlic. "IEE/CSE 598: Lecture 7G (2020-04-15) - Intro. to Spiking Neural Networks and Neuromorphic Computing." On https://www.youtube.com/watch?v=ICw2_49dSNw&ab_channel=TedPavlic

[6] J. Vreeken. "Spiking neural networks, an introduction.", 2003. http://www.ai.jonad.eu/materialy/download/sieci_neuronowe/2003-008.pdf

[7] Cramer, B., Stradmann, Y., Schemmel, J., and Zenke, F. "The Heidelberg spiking datasets for the systematic evaluation of spiking neural networks.", 2019. In https://arxiv.org/abs/1910.07407

[8] André Grüning and Sander M. Bohte. "Spiking Neural Networks: Principles and Challenges." European Symposium on Artificial Neural Networks, Computational Intelligence and Machine Learning, 2014. In https://homepages.cwi.nl/~sbohte/publication/es2014-13Gruning.pdf